In [1]:
import os
import sys
import h5py
import pandas as pd
import dask.dataframe as dd
import gc
import pprint
from scapy.all import *
flags = {
        'R': 0,
        'A': 1,
        'S': 2,
        'DF':3,
        'FA':4,
        'SA':5,
        'RA':6,
        'PA':7,
        'FPA':8
    }

In [2]:
def process_panda(file,counter):
    global flags
    data = {
        'time_delta': [0],
        'ttl':[],
        'len':[],
        'dataofs':[],
        'window':[],
        'seq_delta':[0],
        'ack_delta':[0],
        'flags':[]
    }
    scap = sniff(offline=file,filter='tcp and ip')
    for packet in range(0,len(scap)):
        pkt = scap[packet]
        flag = flags[str(pkt['TCP'].flags)]
        data['ttl'].append(pkt['IP'].ttl)
        data['len'].append(pkt['IP'].len)
        data['dataofs'].append(pkt['TCP'].dataofs)
        data['window'].append(pkt['TCP'].window)
        data['flags'].append(flag)
        data['stream'] = counter
        if packet != 0:
            lst_pkt = scap[packet-1]
            data['time_delta'].append(pkt.time - lst_pkt.time)
            data['seq_delta'].append(pkt['TCP'].seq - lst_pkt['TCP'].seq)
            data['ack_delta'].append(pkt['TCP'].ack - lst_pkt['TCP'].ack)
#     print('seq_delta :',data['seq_delta'])
#     print('ack_delta :',data['ack_delta'])
            
    panda = pd.DataFrame(data=data)
    panda['ttl']=panda['ttl'].astype('float16')
    panda['flags']=panda['flags'].astype('float16')
    panda['dataofs']=panda['dataofs'].astype('float16')
    panda['len']=panda['len'].astype('float16')
    panda['window']=panda['window'].astype('float32')
    panda['seq_delta']=panda['seq_delta'].astype('float32')
    panda['ack_delta']=panda['ack_delta'].astype('float32')
    
    gc.collect()
    return panda

In [3]:
directory = 'dev/streams/'
files = os.listdir(directory)
df = dd.from_pandas(process_panda(directory+files[0],0), npartitions=6)
df.set_index('stream')
files.remove(files[0])
counter=1
for file in files:
    df =  dd.concat([df,process_panda(directory+file,counter)])
    counter+=1
    gc.collect()
df.repartition(npartitions=counter)
print(len(df))


4667


In [4]:
print(df.divisions)

(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non